# Import Modules

In [78]:
import numpy as np
import pandas as pd
import bs4
import requests
import re
from dotenv import load_dotenv
import os
import sqlalchemy as db
from geopy.geocoders import Nominatim, Photon
import folium
import io
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
import pydeck as pdk

%matplotlib inline

# Data Wrangling

### Load .env file

In [35]:
load_dotenv()

True

In [3]:
COOKIES = {'FORLAPDIKTI': os.getenv('FORLAPDIKTI'), 
           'PHPSESSID': os.getenv('PHPSESSID'), 
           'FGTServer': os.getenv('ADRUM')}

BASE_URL = 'https://forlap.kemdikbud.go.id/perguruantinggi/search'

### Fetch universities data from web

In [7]:
data = pd.DataFrame()
PAGE = 0

while True:
    response = requests.get(f'https://forlap.kemdikbud.go.id/perguruantinggi/search/{PAGE}', cookies=COOKIES)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    new_table = pd.read_html(response.text, thousands='.')[0]
    data = pd.concat([data, new_table], ignore_index=True)
    
    if soup.select('div.pagination ul li')[-1].find_all('a') == []:
        break
    else:
        PAGE += 20

### Fetch major data of each universites from web (Cannot be done)

In [8]:
data_prodi = pd.DataFrame()
PAGE = 0

while True:
    response = requests.get(f'https://forlap.kemdikbud.go.id/perguruantinggi/search/{PAGE}', cookies=COOKIES)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    list_univ = [(x.select('td')[1].getText().strip(), 
                  x.select('td')[2].getText(),
                  x.select('a')[0]['href']) 
                  for x in soup.select('tr.ttop') if x.select('td')[3].getText() == 'Prov. Kalimantan Utara']
    for kode, nama, link in list_univ:
        response = requests.get(link)
        prodi_univ = pd.read_html(response.text, thousands='.')[-1]
        prodi_univ[('Kode PT', 'Kode PT')] = kode
        prodi_univ[('Nama PT', 'Nama PT')] = nama
        data_prodi = pd.concat([data_prodi, prodi_univ], ignore_index=True)
    
    if PAGE==20 and soup.select('div.pagination ul li')[-1].find_all('a') == []:
        break
    else:
        PAGE += 20

### Rename columns name and transform dataframe into non-multiindex

In [9]:
first = list(data.columns.get_level_values(1)[:6])
second = list(map(lambda x: x+' 2018/2019', data.columns.get_level_values(1)[6:9]))
third = list(map(lambda x: x+' 2019/2020', data.columns.get_level_values(1)[9:]))

column_names = first + second + third
column_names

['No.',
 'Kode PT',
 'Nama PT',
 'Provinsi',
 'Kategori',
 'Status',
 'Jml Dosen Tetap 2018/2019',
 'Jml Mhs 2018/2019',
 'Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019',
 'Jml Dosen Tetap 2019/2020',
 'Jml Mhs 2019/2020',
 'Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020']

In [10]:
data.columns = data.columns.droplevel(level=0)
data.columns = column_names
data.head(5)

,No.,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
0,1,0,NaN,NaN,NaN,NaN,0,0,1 :,0,0,1 : 0
1,2,1001,Universitas Gadjah Mada,Prov. D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,3,1002,Universitas Indonesia,Prov. D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,4,1003,Universitas Sumatera Utara,Prov. Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,5,1004,Universitas Airlangga,Prov. Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1


### Delete Rows/Columns

In [15]:
# Before deletion
data.shape

(5717, 12)

In [46]:
# After deletion
data.dropna(subset=['Nama PT'], inplace=True)
data.drop(labels='No.', axis=1, inplace=True)
data.shape

(5716, 11)

### Value Reformatting

In [47]:
data.head()

,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
1,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
5,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2


In [40]:
# Delete `Prov.` in Provinsi column
data['Provinsi'] = data['Provinsi'].str.replace('Prov. ', '')
data.head()

,No.,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
1,2,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
2,3,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
3,4,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
4,5,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
5,6,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2


# Data Storing

In [4]:
engine = db.create_engine(f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}")

In [70]:
with engine.connect() as conn:
    data.to_sql('universitas', con=conn, index=False)

# Data Fetching from Database

In [5]:
engine = db.create_engine(f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}")

In [6]:
meta = db.MetaData(bind=engine)
meta.reflect()

In [7]:
univ = db.Table('universitas', meta, autoload=True, autoload_with=engine)

In [8]:
query = db.select([univ])
with engine.connect() as conn:
    result = conn.execute(query).fetchall()
len(result)

5716

In [186]:
data_univ = pd.DataFrame.from_records(result, columns=univ.c.keys())
data_univ

,index,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
0,1,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
1,2,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
2,3,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
3,4,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
4,5,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2
...,...,...,...,...,...,...,...,...,...,...,...,...
5711,5712,495003,Politeknik STIA LAN Makassar,Sulawesi Selatan,Negeri,Aktif,45,550,1 : 12.2,45,805,1 : 17.9
5712,5713,504001,Akademi Metrologi dan Instrumentasi,Jawa Barat,None,Aktif,9,165,1 : 18.3,9,165,1 : 18.3
5713,5714,513001,Sekolah Tinggi Intelijen Negara,Jawa Barat,None,Aktif,22,0,1 : 0,22,0,1 : 0
5714,5715,525001,Politeknik Pekerjaan Umum,Jawa Tengah,None,Aktif,23,0,1 : 0,23,137,1 : 6


In [187]:
data = data_univ[data_univ['Status'] == 'Aktif']
data

,index,Kode PT,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020
0,1,1001,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8
1,2,1002,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5
2,3,1003,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4
3,4,1004,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1
4,5,1005,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2
...,...,...,...,...,...,...,...,...,...,...,...,...
5711,5712,495003,Politeknik STIA LAN Makassar,Sulawesi Selatan,Negeri,Aktif,45,550,1 : 12.2,45,805,1 : 17.9
5712,5713,504001,Akademi Metrologi dan Instrumentasi,Jawa Barat,None,Aktif,9,165,1 : 18.3,9,165,1 : 18.3
5713,5714,513001,Sekolah Tinggi Intelijen Negara,Jawa Barat,None,Aktif,22,0,1 : 0,22,0,1 : 0
5714,5715,525001,Politeknik Pekerjaan Umum,Jawa Tengah,None,Aktif,23,0,1 : 0,23,137,1 : 6


# Exploratory Data Analysis

In [188]:
nominatim = Nominatim(user_agent='id-state')
photon = Photon(user_agent='id_state')

In [189]:
def get_location(place):
    loc = photon.geocode(place)
    try:
        return loc.latitude, loc.longitude
    except:
        return None

In [190]:
def save_img(m, filename):
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(filename)

### Peta Distribusi Jumlah Perguruan Tinggi di Tiap Provinsi

In [191]:
byProv = data.groupby('Provinsi').count()
byProv['Location'] = [get_location(prov) for prov in byProv.index]
byProv.Location

Provinsi
Aceh                                     (4.3685491, 97.0253024)
Bali                            (-8.23565805, 115.1223865976744)
Bangka Belitung                        (-2.7052886, 106.3585607)
Banten                         (-6.47800285, 105.54102849016589)
Bengkulu                               (-3.5186763, 102.5359834)
D.I. Yogyakarta        (-7.9778383999999996, 110.36722565020224)
D.K.I. Jakarta                          (-6.1753942, 106.827183)
Gorontalo                               (0.7186174, 122.4555927)
Jambi                           (-1.6454854, 102.93517597904014)
Jawa Barat                             (-6.8891904, 107.6404716)
Jawa Tengah                     (-5.6259648, 110.37164897312938)
Jawa Timur                             (-7.6977397, 112.4914199)
Kalimantan Barat       (-1.2003376000000001, 109.93193384532759)
Kalimantan Selatan                     (-2.9285686, 115.3700718)
Kalimantan Tengah                       (-1.499583, 113.2903307)
Kalimantan Timur

In [192]:
m = folium.Map(location=get_location('Indonesia'), zoom_start=5, tiles='CartoDB dark_matter', min_zoom=5, max_zoom=5)
for prov in byProv.index:
    folium.CircleMarker(location=byProv.loc[prov, 'Location'], 
                        popup=f'''<strong>{prov}</strong>\
                        <br><br>\
                        {byProv.loc[prov, 'Kode PT']}''',
                        radius=byProv.loc[prov, 'Kode PT']/10,
                        stroke=False, fill=True, fill_opacity=0.5
                       ).add_to(m)

m.save('pt_per_prov.html')
# save_img(m, 'pt_per_prov.png')
m

### Peta Distribusi Jumlah Mahasiswa di Tiap Provinsi - 2019

In [193]:
byProv_mahasiswa = data.groupby('Provinsi').sum()
byProv_mahasiswa['Location'] = [get_location(prov) for prov in byProv.index]
byProv_mahasiswa.Location

Provinsi
Aceh                                     (4.3685491, 97.0253024)
Bali                            (-8.23565805, 115.1223865976744)
Bangka Belitung                        (-2.7052886, 106.3585607)
Banten                         (-6.47800285, 105.54102849016589)
Bengkulu                               (-3.5186763, 102.5359834)
D.I. Yogyakarta        (-7.9778383999999996, 110.36722565020224)
D.K.I. Jakarta                          (-6.1753942, 106.827183)
Gorontalo                               (0.7186174, 122.4555927)
Jambi                           (-1.6454854, 102.93517597904014)
Jawa Barat                             (-6.8891904, 107.6404716)
Jawa Tengah                     (-5.6259648, 110.37164897312938)
Jawa Timur                             (-7.6977397, 112.4914199)
Kalimantan Barat       (-1.2003376000000001, 109.93193384532759)
Kalimantan Selatan                     (-2.9285686, 115.3700718)
Kalimantan Tengah                       (-1.499583, 113.2903307)
Kalimantan Timur

In [194]:
jmlMahasiswa = byProv_mahasiswa[['Jml Mhs 2019/2020']]
norm = plt.Normalize(jmlMahasiswa.min(), jmlMahasiswa.max())
jmlMahasiswa['radius'] = jmlMahasiswa['Jml Mhs 2019/2020'].map(lambda x: norm(x)*100)
jmlMahasiswa['radius_log'] = jmlMahasiswa['Jml Mhs 2019/2020'].map(lambda x: np.log(x))

jmlMahasiswa.head()

<ipython-input-194-cd13a954d8f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jmlMahasiswa['radius'] = jmlMahasiswa['Jml Mhs 2019/2020'].map(lambda x: norm(x)*100)
<ipython-input-194-cd13a954d8f4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jmlMahasiswa['radius_log'] = jmlMahasiswa['Jml Mhs 2019/2020'].map(lambda x: np.log(x))


,Jml Mhs 2019/2020,radius,radius_log
Provinsi,,,
Aceh,208608,14.811341,12.248212
Bali,172193,12.063441,12.056371
Bangka Belitung,15789,0.261094,9.667069
Banten,1337523,100.000000,14.106330
Bengkulu,57751,3.427574,10.963896


In [195]:
m = folium.Map(location=get_location('Indonesia'), zoom_start=5, tiles='CartoDB dark_matter', min_zoom=5, max_zoom=5)
for prov in byProv_mahasiswa.index:
    folium.CircleMarker(location=byProv_mahasiswa.loc[prov, 'Location'], 
                        popup=f'''<strong>{prov}</strong>\
                        <br><br>\
                        {byProv_mahasiswa.loc[prov, 'Kode PT']/10**6:.2f} juta''',
                        radius=jmlMahasiswa.loc[prov, 'radius'],
                        stroke=False, fill=True, fill_opacity=0.5, fill_color='#fca311'
                       ).add_to(m)

m.save('mhs_per_prov.html')
# save_img(m, 'mhs_per_prov.png')
m

### Peta Distribusi Jumlah Dosen di Tiap Provinsi - 2019

In [196]:
byProv_dosen = byProv_mahasiswa.copy()
byProv_dosen.head()

,index,Kode PT,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Location
Provinsi,,,,,,,
Aceh,595084,23096974,8427,218077,8427,208608,"(4.3685491, 97.0253024)"
Bali,229826,8538782,7075,147329,7074,172193,"(-8.23565805, 115.1223865976744)"
Bangka Belitung,25480,1222584,724,13937,724,15789,"(-2.7052886, 106.3585607)"
Banten,398012,16479017,11462,1309665,11449,1337523,"(-6.47800285, 105.54102849016589)"
Bengkulu,52679,2494031,2600,59824,2600,57751,"(-3.5186763, 102.5359834)"


In [197]:
jmlDosen = byProv_dosen[['Jml Dosen Tetap 2019/2020']]
norm = plt.Normalize(jmlDosen.min(), jmlDosen.max())
jmlDosen['radius'] = jmlDosen['Jml Dosen Tetap 2019/2020'].map(lambda x: norm(x)*100)

jmlDosen.sample(5)

<ipython-input-197-4bd7ae73d21e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jmlDosen['radius'] = jmlDosen['Jml Dosen Tetap 2019/2020'].map(lambda x: norm(x)*100)


,Jml Dosen Tetap 2019/2020,radius
Provinsi,,
Sulawesi Utara,4839,11.351051
D.K.I. Jakarta,28740,74.680056
Kepulauan Riau,2308,4.644816
Maluku,2792,5.927241
Kalimantan Utara,555,0.000000


In [213]:
m = folium.Map(location=get_location('Indonesia'), zoom_start=5, tiles='CartoDB dark_matter', min_zoom=5, max_zoom=5)
for prov in byProv_mahasiswa.index:
    folium.CircleMarker(location=byProv_dosen.loc[prov, 'Location'], 
                        popup=f'''<strong>{prov}</strong>\
                        <br><br>\
                        {byProv_dosen.loc[prov, 'Kode PT']/10**6:.2f} juta''',
                        radius=jmlDosen.loc[prov, 'radius'],
                        stroke=False, fill=True, fill_opacity=0.5, fill_color='#f72585'
                       ).add_to(m)

m.save('dosen_per_prov.html')
# save_img(m, 'dosen_per_prov.png')
m

### Peta Distribusi Rasio Dosen dan Mahasiswa - 2019

In [215]:
data_ratio = data_univ[['Provinsi', 'Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020']].copy()
data_ratio.rename(columns={'Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020':'Rasio'}, inplace=True)
data_ratio['Rasio'] = data_ratio['Rasio'].map(lambda x: float(x.split(':')[-1]))

data_ratio

,Provinsi,Rasio
0,D.I. Yogyakarta,20.8
1,D.K.I. Jakarta,17.5
2,Sumatera Utara,25.4
3,Jawa Timur,20.1
4,Sulawesi Selatan,18.2
...,...,...
5711,Sulawesi Selatan,17.9
5712,Jawa Barat,18.3
5713,Jawa Barat,0.0
5714,Jawa Tengah,6.0


In [216]:
byProv_ratio = data_ratio.groupby('Provinsi').median()
byProv_ratio['Location'] = [get_location(prov) for prov in byProv_ratio.index]
byProv_ratio

,Rasio,Location
Provinsi,,
Aceh,11.60,"(4.3685491, 97.0253024)"
Bali,8.40,"(-8.23565805, 115.1223865976744)"
Bangka Belitung,4.40,"(-2.7052886, 106.3585607)"
Banten,9.40,"(-6.47800285, 105.54102849016589)"
Bengkulu,6.30,"(-3.5186763, 102.5359834)"
D.I. Yogyakarta,15.20,"(-7.9778383999999996, 110.36722565020224)"
D.K.I. Jakarta,7.25,"(-6.1753942, 106.827183)"
Gorontalo,13.40,"(0.7186174, 122.4555927)"
Jambi,7.95,"(-1.6454854, 102.93517597904014)"


In [217]:
avg_ratio = byProv_ratio[['Rasio']]
norm = plt.Normalize(avg_ratio.min(), avg_ratio.max())
avg_ratio['radius'] = avg_ratio['Rasio'].map(lambda x: norm(x)*50)

avg_ratio

,Rasio,radius
Provinsi,,
Aceh,11.60,23.716012
Bali,8.40,14.048338
Bangka Belitung,4.40,1.963746
Banten,9.40,17.069486
Bengkulu,6.30,7.703927
D.I. Yogyakarta,15.20,34.592145
D.K.I. Jakarta,7.25,10.574018
Gorontalo,13.40,29.154079
Jambi,7.95,12.688822


In [218]:
m = folium.Map(location=get_location('Indonesia'), zoom_start=5, tiles='CartoDB dark_matter', min_zoom=5, max_zoom=5)
for prov in byProv_mahasiswa.index:
    folium.CircleMarker(location=byProv_ratio.loc[prov, 'Location'], 
                        popup=f'''<strong>{prov}</strong>\
                        <br><br>\
                        {byProv_dosen.loc[prov, 'Kode PT']/10**6:.2f} juta''',
                        radius=avg_ratio.loc[prov, 'radius'],
                        stroke=False, fill=True, fill_opacity=0.5, fill_color='#00f5d4'
                       ).add_to(m)

m.save('ratio_per_prov.html')
# save_img(m, 'ratio_per_prov.png')
m

### Peta Distribusi Mahasiswa - 2019

In [245]:
growth = data.iloc[:, 2:]
growth['Dosen'] = growth['Jml Dosen Tetap 2019/2020'] - growth['Jml Dosen Tetap 2018/2019']
growth['Mahasiswa'] = growth['Jml Mhs 2019/2020'] - growth['Jml Mhs 2018/2019']

growth

,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020,Dosen,Mahasiswa
0,Universitas Gadjah Mada,D.I. Yogyakarta,Negeri,Aktif,2703,56200,1 : 20.8,2703,56142,1 : 20.8,0,-58
1,Universitas Indonesia,D.K.I. Jakarta,Negeri,Aktif,2273,46248,1 : 20.3,2273,39826,1 : 17.5,0,-6422
2,Universitas Sumatera Utara,Sumatera Utara,Negeri,Aktif,1633,44432,1 : 27.2,1633,41468,1 : 25.4,0,-2964
3,Universitas Airlangga,Jawa Timur,Negeri,Aktif,1779,36253,1 : 20.4,1779,35766,1 : 20.1,0,-487
4,Universitas Hasanuddin,Sulawesi Selatan,Negeri,Aktif,1909,31396,1 : 16.4,1909,34787,1 : 18.2,0,3391
...,...,...,...,...,...,...,...,...,...,...,...,...
5711,Politeknik STIA LAN Makassar,Sulawesi Selatan,Negeri,Aktif,45,550,1 : 12.2,45,805,1 : 17.9,0,255
5712,Akademi Metrologi dan Instrumentasi,Jawa Barat,None,Aktif,9,165,1 : 18.3,9,165,1 : 18.3,0,0
5713,Sekolah Tinggi Intelijen Negara,Jawa Barat,None,Aktif,22,0,1 : 0,22,0,1 : 0,0,0
5714,Politeknik Pekerjaan Umum,Jawa Tengah,None,Aktif,23,0,1 : 0,23,137,1 : 6,0,137


In [248]:
growth.sort_values('Dosen')

,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020,Dosen,Mahasiswa
4165,IAIN Walisongo semarang,Jawa Tengah,Negeri,Aktif,355,4294,1 : 12.1,0,0,1 : 0,-355,-4294
5432,Institut Agama Hindu Negeri Gde Pudja Mataram,Nusa Tenggara Barat,Negeri,Aktif,88,690,1 : 7.8,0,0,1 : 0,-88,-690
1818,STMIK Amikom,D.I. Yogyakarta,Swasta,Aktif,52,2272,1 : 43.7,0,0,1 : 0,-52,-2272
4742,"STAI Darussalam Martapura, Kalimantan Selatan",Kalimantan Selatan,Swasta,Aktif,47,683,1 : 14.5,0,0,1 : 0,-47,-683
3122,Sekolah Tinggi Ilmu Pertanian Yapim Maros,Sulawesi Selatan,Swasta,Aktif,38,437,1 : 11.5,0,0,1 : 0,-38,-437
...,...,...,...,...,...,...,...,...,...,...,...,...
2021,STMIK Sinar Nusantara,Jawa Tengah,Swasta,Aktif,43,1571,1 : 36.5,43,1341,1 : 31.2,0,-230
2020,Sekolah Tinggi Ilmu Ekonomi Rajawali,Jawa Tengah,Swasta,Aktif,15,402,1 : 26.8,15,410,1 : 27.3,0,8
2019,Sekolah Tinggi Ilmu Perikanan Kalinyamat,Jawa Tengah,Swasta,Aktif,1,0,1 : 0,1,0,1 : 0,0,0
2026,Sekolah Tinggi Ilmu Administrasi Madani,Jawa Tengah,Swasta,Aktif,17,266,1 : 15.6,17,298,1 : 17.5,0,32


In [250]:
growth.sort_values('Mahasiswa')

,Nama PT,Provinsi,Kategori,Status,Jml Dosen Tetap 2018/2019,Jml Mhs 2018/2019,Rasio Dosen Tetap/Jumlah Mahasiswa 2018/2019,Jml Dosen Tetap 2019/2020,Jml Mhs 2019/2020,Rasio Dosen Tetap/Jumlah Mahasiswa 2019/2020,Dosen,Mahasiswa
28,Universitas Halu Oleo,Sulawesi Tenggara,Negeri,Aktif,1503,53900,1 : 35.9,1503,43119,1 : 28.7,0,-10781
806,Universitas Bina Sarana Informatika,D.K.I. Jakarta,Swasta,Aktif,1141,43416,1 : 38.1,1141,33802,1 : 29.6,0,-9614
8,Universitas Sriwijaya,Sumatera Selatan,Negeri,Aktif,1371,39925,1 : 29.1,1371,30787,1 : 22.5,0,-9138
4141,Universitas Islam Negeri Syarif Hidayatullah,D.K.I. Jakarta,Negeri,Aktif,1022,33919,1 : 33.2,1022,26208,1 : 25.6,0,-7711
2957,Universitas Indonesia Timur,Sulawesi Selatan,Swasta,Aktif,306,19942,1 : 65.2,306,12900,1 : 42.2,0,-7042
...,...,...,...,...,...,...,...,...,...,...,...,...
4148,Universitas Islam Negeri Sumatera Utara Medan,Sumatera Utara,Negeri,Aktif,510,15898,1 : 31.2,510,26724,1 : 52.4,0,10826
4154,Universitas Islam Negeri Raden Intan Lampung,Lampung,Negeri,Aktif,419,28833,1 : 68.8,419,40230,1 : 96,0,11397
12,Universitas Udayana,Bali,Negeri,Aktif,1689,30741,1 : 18.2,1689,47958,1 : 28.4,0,17217
30,Universitas Terbuka,Banten,Negeri,Aktif,679,1022259,1 : 1505.5,679,1045665,1 : 1540,0,23406


### Peta Distribusi Dosen - 2019

### Peta Rasio Dosen dan Mahasiswa - 2019

### Peta Pertumbuhan Jumlah Mahasiswa (Positif, Negatif)